In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [4]:
# iv: image option
length = 512
channel = 96
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [5]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
eeg_dataset, splits_path, splits_shuffled_path = os.listdir(torch_models_dir)

In [6]:
eeg_dataset = os.path.join(torch_models_dir, eeg_dataset)
splits_path = os.path.join(torch_models_dir, splits_path)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_path)

/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth 
 /media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth


In [7]:
opt = {
    # Dataset options
    "iv": "image",
    "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "run": "none",
    "eeg_dataset": eeg_dataset,
    "splits_path": splits_path,
    "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"incremental",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.iv)

image


In [8]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from data_loader import EEGDataset, Splitter, SplitterWithData
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

In [9]:
def load_dataset(iv,
             offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-4) - 5 fold cross validation
             total,
             classes,
             classifier,
             batch_size,
             GPUindex,
             length, # 512
             channel, # 96
             min_CNN,
             opt,
             kind):        
    # Load dataset
    dataset = EEGDataset(iv, eeg_dataset, classifier, map_idx = None)
    print("DONE: LOAD DATASET")
    # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
    if kind=="from-scratch":
        relabel = False
    if kind=="incremental":
        relabel = False
    if kind=="no-model-file":
        relabel = True
    splitter = {split: SplitterWithData(iv,
                    dataset,
                    splits_path,
                    classes,
                    split_num,
                    split,
                    relabel) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [10]:
# CNN network
opt.classifier = "CNN"
opt.batch_size = 16
opt.kind = "from-scratch"
opt.run = "imagenet40-1000"
opt.fold = 5

In [12]:
dataset, loaders, splitter = load_dataset(opt.iv,
                             opt.offset,
                             opt.eeg_dataset,
                             opt.splits_path,
                             0, #split_num
                             n_classes,
                             classes,
                             opt.classifier,
                             opt.batch_size,
                             opt.GPUindex,
                             length,
                             channel,
                             min_CNN,
                             opt,
                             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [13]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 32000 idx, val: 4000 idx, test: 4000 idx}
# Each loader batch has {train: 2000 idx, val: 250idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i in range(0, 40):
    eeg, label_val = splitter["val"][i*100]
    eeg, label_train = splitter["train"][i*800]
    print(f"{i+1}: Label val: {label_val}; label train: {label_train}")
# print(splitter["val"].split_idx[:10])

<class 'data_loader.EEGDataset'>
<class 'dict'>
3 [2000, 250, 250]
1: Label val: 0; label train: 0
2: Label val: 17; label train: 17
3: Label val: 28; label train: 28
4: Label val: 7; label train: 7
5: Label val: 33; label train: 33
6: Label val: 12; label train: 12
7: Label val: 21; label train: 21
8: Label val: 3; label train: 3
9: Label val: 25; label train: 25
10: Label val: 36; label train: 36
11: Label val: 10; label train: 10
12: Label val: 15; label train: 15
13: Label val: 19; label train: 19
14: Label val: 31; label train: 31
15: Label val: 23; label train: 23
16: Label val: 5; label train: 5
17: Label val: 38; label train: 38
18: Label val: 8; label train: 8
19: Label val: 1; label train: 1
20: Label val: 34; label train: 34
21: Label val: 29; label train: 29
22: Label val: 26; label train: 26
23: Label val: 13; label train: 13
24: Label val: 11; label train: 11
25: Label val: 22; label train: 22
26: Label val: 18; label train: 18
27: Label val: 6; label train: 6
28: Label v

In [11]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1, 96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_CNN(
  (conv1): Conv1d(1, 8, kernel_size=(32,), stride=(1,))
  (fc1): Linear(in_features=768, out_features=40, bias=True)
  (pool1): AvgPool1d(kernel_size=(128,), stride=(64,), padding=(0,))
  (activation): ELU(alpha=1.0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=240, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_CNN                           [1, 40]                   --
├─Conv1d: 1-1                            [96, 8, 481]              264
├─ELU: 1-2                               [96, 8, 481]              --
├─Dropout: 1-3                           [1, 481, 768]             --
├─Linear: 1-4                            [1, 481, 40]              30,760
├─Dropout: 1-5                           [1, 481, 40]              --
├─AvgPool1d: 1-6                         [1, 40, 6]                --
├─Linear: 1-7                            [1, 40]                   9,640
Total params: 40,664
Trainable params: 40,664
Non-trainable params: 0
Total mult-adds (M): 12.23
Input size (MB): 0.20
Forward/backward pass size (MB): 3.11
Params size (MB): 0.16
Estimated Total Size (MB): 3.47

In [14]:
model_path = (opt.iv+
                  "-"+
                  opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel)+
                  "-"+
                  str(0)) + "-noshuffle"
                  
channel_idx=None
print(model_path)

image-CNN-512-96-0-noshuffle


In [15]:
print(opt)

Namespace(iv='image', offset=None, results_file='results.pkl', subject=0, run='imagenet40-1000', eeg_dataset='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth', splits_path='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth', fold=5, batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='CNN')


In [16]:
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

epoch 1
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.674734592437744; accuracy=0.02512500062584877
Batch 2000: Loss=3.6773087978363037; accuracy=0.028468752279877663
Validation accuracy: 0.03475000336766243
epoch 2
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6136465072631836; accuracy=0.03750000149011612
Batch 2000: Loss=3.646854877471924; accuracy=0.037312500178813934
Validation accuracy: 0.04125000163912773
epoch 3
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.689420700073242; accuracy=0.0416250005364418
Batch 2000: Loss=3.531879425048828; accuracy=0.043656252324581146
Validation accuracy: 0.04175000265240669
epoch 4
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 

Validation accuracy: 0.039750002324581146
epoch 29
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.4717419147491455; accuracy=0.06706250458955765
Batch 2000: Loss=3.7850100994110107; accuracy=0.06365625560283661
Validation accuracy: 0.03700000047683716
epoch 30
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.625499725341797; accuracy=0.060875002294778824
Batch 2000: Loss=3.496779203414917; accuracy=0.061375003308057785
Validation accuracy: 0.039750002324581146
epoch 31
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.78896164894104; accuracy=0.06450000405311584
Batch 2000: Loss=3.7062106132507324; accuracy=0.06328125298023224
Validation accuracy: 0.04700000211596489
epoch 32
Size of input:  torch.Size([16, 96, 512])
Size of target:  tor

Batch 2000: Loss=3.462477684020996; accuracy=0.06725000590085983
Validation accuracy: 0.04450000077486038
epoch 57
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.5818026065826416; accuracy=0.07193750143051147
Batch 2000: Loss=3.6571383476257324; accuracy=0.06659375131130219
Validation accuracy: 0.04000000283122063
epoch 58
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.454103469848633; accuracy=0.06818750500679016
Batch 2000: Loss=3.676274299621582; accuracy=0.06625000387430191
Validation accuracy: 0.0442500002682209
epoch 59
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.461456775665283; accuracy=0.06925000250339508
Batch 2000: Loss=3.4073843955993652; accuracy=0.06556250154972076
Validation accuracy: 0.04075000062584877
epoch 60
S

Batch 2000: Loss=3.750143527984619; accuracy=0.06965625286102295
Validation accuracy: 0.04200000315904617
epoch 85
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=4.592222213745117; accuracy=0.06781250238418579
Batch 2000: Loss=3.506661891937256; accuracy=0.06606250256299973
Validation accuracy: 0.04400000348687172
epoch 86
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.5865254402160645; accuracy=0.07106250524520874
Batch 2000: Loss=3.388021469116211; accuracy=0.06737500429153442
Validation accuracy: 0.04200000315904617
epoch 87
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.8367867469787598; accuracy=0.07231250405311584
Batch 2000: Loss=3.617060661315918; accuracy=0.06956250220537186
Validation accuracy: 0.04150000214576721
epoch 88
S

In [17]:
val =accuracy_val
test = accuracy_test

print("Validation accuracy: ", val)
print("Test accuracy: ", test)

Validation accuracy:  0.042750000953674316
Test accuracy:  0


In [ ]:
print(accuracy_val)